In [2]:
import re
import pandas as pd



In [3]:

#df = pd.read_csv('../datasets/deeploc_lookup_labels.txt', header=None, names=['ID', 'Location'])
df = pd.read_csv('../datasets/la_query_setHARD_labels.txt', header=None, names=['ID', 'Location'])

df

,ID,Location
0,Q12981,Endoplasmic.reticulum
1,Q8R0A6,Extracellular
2,Q9NZ09,Cytoplasm
3,Q92558,Cytoplasm
4,Q95XQ1,Nucleus
...,...,...
485,Q9H9E3,Cytoplasm
486,Q9FFS4,Nucleus
487,Q62824,Cell.membrane
488,Q6ZWX6,Cytoplasm


In [4]:


with open('../datasets/similar.txt', 'r') as file:
    lines = file.readlines()

families = []
protid_list = []

protid_pattern = re.compile(r'\((\S+)\)')  # Matches protein IDs inside parentheses

current_family = None
for line in lines:
    # Check for family name (line containing 'family' or similar)
    if 'family' in line.lower():  # Adjust the condition to match family names
        current_family = line.strip()  # Capture the entire line as the family name
    elif current_family:
        # Search for protein IDs in the current family section
        protid_matches = protid_pattern.findall(line)
        if protid_matches:
            for protid in protid_matches:
                families.append(current_family)
                protid_list.append(protid)

family_df = pd.DataFrame({
    'Family_raw': families,
    'ID': protid_list
})

family_df


,Family_raw,ID
0,'GDSL' lipolytic enzyme family,Q3MKY2
1,'GDSL' lipolytic enzyme family,P86830
2,'GDSL' lipolytic enzyme family,B4FZ87
3,'GDSL' lipolytic enzyme family,Q9LU14
4,'GDSL' lipolytic enzyme family,P40602
...,...,...
519755,[NiFe]/[NiFeSe] hydrogenase small subunit family,Q00406
519756,[NiFe]/[NiFeSe] hydrogenase small subunit family,Q50248
519757,[NiFe]/[NiFeSe] hydrogenase small subunit family,Q50225
519758,[NiFe]/[NiFeSe] hydrogenase small subunit family,Q58591


In [5]:
merged_df = pd.merge(df, family_df, on='ID', how='left')
merged_df

,ID,Location,Family_raw
0,Q12981,Endoplasmic.reticulum,SEC20 family
1,Q8R0A6,Extracellular,NaN
2,Q9NZ09,Cytoplasm,NaN
3,Q92558,Cytoplasm,SCAR/WAVE family
4,Q95XQ1,Nucleus,NaN
...,...,...,...
487,Q9H9E3,Cytoplasm,COG4 family
488,Q9FFS4,Nucleus,DDA1 family
489,Q62824,Cell.membrane,SEC8 family
490,Q6ZWX6,Cytoplasm,eIF-2-alpha family


In [6]:

merged_df['family'] = merged_df['Family_raw']
merged_df['sub_family'] = pd.NA  # Initialize 'sub_family' with NaN

for index, row in merged_df.iterrows():
    if 'y.' in str(row['Family_raw']):  # If period is detected in 'Family_raw'
        merged_df.at[index, 'family'], merged_df.at[index, 'sub_family'] = row['Family_raw'].split('.', 1)
    else:
        merged_df.at[index, 'family'] = row['Family_raw']  # No change to family if no period
merged_df

,ID,Location,Family_raw,family,sub_family
0,Q12981,Endoplasmic.reticulum,SEC20 family,SEC20 family,<NA>
1,Q8R0A6,Extracellular,NaN,NaN,<NA>
2,Q9NZ09,Cytoplasm,NaN,NaN,<NA>
3,Q92558,Cytoplasm,SCAR/WAVE family,SCAR/WAVE family,<NA>
4,Q95XQ1,Nucleus,NaN,NaN,<NA>
...,...,...,...,...,...
487,Q9H9E3,Cytoplasm,COG4 family,COG4 family,<NA>
488,Q9FFS4,Nucleus,DDA1 family,DDA1 family,<NA>
489,Q62824,Cell.membrane,SEC8 family,SEC8 family,<NA>
490,Q6ZWX6,Cytoplasm,eIF-2-alpha family,eIF-2-alpha family,<NA>


In [7]:
#merged_df.to_csv('../datasets/deeploc_lookup_labels_family.csv', index=False)
merged_df.to_csv('../datasets/la_query_setHARD_labels_family.csv', index=False)

